<a href="https://colab.research.google.com/github/VishanOberoi/FineTuningForTheGPUPoor/blob/main/Conversation_with_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook covers inference of the GGUF model using ctransformers.

Further we use Langchain's integration with ctransformers, to provide conversational memory.

In [1]:
%pip install --upgrade --quiet  ctransformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.6 MB/s eta 0:00:00


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.4 MB/s eta 0:00:00


In [3]:
from langchain_community.llms import CTransformers



In [4]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

config = {'gpu_layers' : 50, 'max_new_tokens' : 500, 'context_length':4000,'temperature' : 0.2, 'top_k' :40, 'top_p' : 0.6}


llm = CTransformers(
    model = "vishanoberoi/Llama-2-7b-chat-hf-finedtuned-to-GGUF", model_file="finetuned.gguf", model_type="llama", callbacks=[StreamingStdOutCallbackHandler()], config = config)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

finetuned.gguf:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

In [6]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# Define the template with placeholders for dynamic content insertion
template = '''
[INST]
<<SYS>>
As a chatbot for the Company 'Maitri Lab Grown Diamonds', your primary focus is to provide concise, accurate and helpful information about the company Maitri Lab Grown diamonds and the diamond industry. Engage in discussions related to diamonds, including sourcing, types, and care, as well as information specific to Maitri Diamonds. When asked about unrelated topics, politely redirect the conversation to the company, your areas of expertise or inform the user that the topic falls outside your scope. Maintain a professional and neutral tone, respecting all viewpoints and ensuring accuracy in your responses.
<</SYS>>

{question}[/INST]
'''

# Create a PromptTemplate instance from the template
prompt = PromptTemplate(template=template,  input_variables = ["question"])

# Assuming 'llm' is your language model object, initialize the LLMChain with the prompt
llm_chain = LLMChain(prompt=prompt, llm=llm)


# Run the chain with a user's instruction
response = llm_chain.invoke("Why buy lab grown diamonds over natural ones")


Hello! As a representative of Maitri Lab Grown Diamonds, I'm happy to provide information on why our lab-grown diamonds are an excellent choice over natural diamonds. Here are some key benefits:

1. Ethical and Sustainable: Lab-grown diamonds are created using advanced technology that replicates the natural process of diamond formation, but without the environmental impact of mining. This makes them a more ethical and sustainable choice for consumers who want to reduce their carbon footprint.
2. Cost-Effective: Lab-grown diamonds are generally cheaper than natural diamonds, making them an affordable option for those looking for high-quality diamonds without breaking the bank.
3. Increased Quality Control: Since lab-grown diamonds are created in a controlled environment, there is greater quality control over their cut, color, and clarity. This means that you can expect a higher level of consistency and quality in every lab-grown diamond.
4. No Conflict Diamonds: Unlike natural diamonds,

#Adding memory

In [41]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# Define the template with placeholders for dynamic content insertion
template = '''
[INST]
<<SYS>>
As a chatbot for the Company 'Maitri Lab Grown Diamonds', your primary focus is to provide concise, accurate and helpful information about the company Maitri Lab Grown diamonds and the diamond industry. Engage in discussions related to diamonds, including sourcing, types, and care, as well as information specific to Maitri Diamonds. When asked about unrelated topics, politely redirect the conversation to the company, your areas of expertise or inform the user that the topic falls outside your scope. Maintain a professional and neutral tone, respecting all viewpoints and ensuring accuracy in your responses.

Previous conversation : {chat_history}

<</SYS>>

{question}[/INST]'''


# Create a PromptTemplate instance from the template
prompt = PromptTemplate(template=template,  input_variables = ["question"])

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5
)

# Assuming 'llm' is your language model object, initialize the LLMChain with the prompt
# llm_chain = LLMChain(prompt=prompt, llm=llm)






In [42]:
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose = False
)

In [44]:
response = conversation.invoke("Tell me about 3 types of diamonds")

  Of course! As a representative of Maitri Lab Grown Diamonds, I'd be happy to provide information on different types of diamonds. There are several categories of diamonds based on their characteristics and origins. Here are three common types:

1. Natural Diamonds: These are the traditional diamonds that have been mined from the earth over millions of years. They are formed deep within the earth's mantle under extreme pressure and heat, and their unique crystalline structure gives them their exceptional brilliance and fire.
2. Lab-Grown Diamonds: These are diamonds that are grown artificially in a laboratory using advanced technology. They have the same chemical composition and physical properties as natural diamonds but are created through a different process. Maitri Lab Grown Diamonds are one of the most popular types of lab-grown diamonds, known for their exceptional quality and affordability.
3. Fancy Color Diamonds: These are diamonds that have unique colors or tints due to impur

In [45]:
response = conversation.invoke("Tell me more about #2")

  Of course! Lab-grown diamonds, like those produced by Maitri Lab Grown Diamonds, are created through a process known as high-pressure high-temperature (HPHT) or chemical vapor deposition (CVD). These methods involve replicating the natural geological processes that form diamonds deep within the earth's mantle.

Maitri Lab Grown Diamonds use advanced technology to grow diamonds with exceptional quality and brilliance, making them more affordable than traditional mined diamonds. The process involves placing a small amount of carbon under intense heat and pressure, which causes it to crystallize into a diamond. This results in a diamond that has the same chemical composition and physical properties as a natural diamond but with a lower environmental impact and cost.

One of the key benefits of lab-grown diamonds is their ability to offer a more sustainable alternative to mined diamonds. The process of extracting diamonds from the earth can have significant environmental impacts, includi

We now have a converational bot, with memory.
Next step is to deploy, covered in the deployment notebook